In [1]:
import numpy as np
import pandas as pd
from os import path
from random import choice, shuffle

from ga4stpg.graph import Graph, ReaderORLibrary
from ga4stpg.graph.steiner import (prunning_mst, shortest_path,
                            shortest_path_origin_prim,
                            prunning_kruskal_mst,
                            shortest_path_with_origin)

from ga4stpg.graph.util import (is_steiner_tree, 
                                has_cycle)


In [2]:
from ga4stpg.tree.generate import GenerateBasedPrimRST
from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.pxpartition import PartitionCrossoverSteinerTree as PXST
from ga4stpg.tree.mutate import Prunning

In [3]:
# input
input_file = 'steinc5.txt'
folder_datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(folder_datasets, input_file)
print(filename)

assert path.exists(filename)

reader = ReaderORLibrary()

stpg = reader.parser(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinc5.txt


In [4]:
evaluator = EvaluateTreeGraph(stpg)
generator = GenerateBasedPrimRST(stpg)
crossover = PXST(stpg)
prunner   = Prunning(stpg)

In [5]:
%%time

data = list()
data2 = list()
print(stpg.nro_nodes)

for _ in range(50):
    
    aa = prunner(generator())
    is_aa_st, response = is_steiner_tree(aa, stpg)
    aa_cost, _ = evaluator(aa)
    response['origem'] = 'a'
    response['is_st'] = is_aa_st
    data2.append(response)

    
    bb = prunner(generator())
    is_bb_st, response = is_steiner_tree(bb, stpg)
    bb_cost, _ = evaluator(bb)
    response['origem'] = 'b'
    response['is_st'] = is_bb_st
    data2.append(response)
    
    child_c, child_d = crossover(aa, bb)
    
    is_cc_st, response = is_steiner_tree(child_c, stpg)
    cc_cost, _ = evaluator(child_c)
    response['origem'] = 'c'
    response['is_st'] = is_cc_st
    data2.append(response)

    is_dd_st, response = is_steiner_tree(child_d, stpg)
    dd_cost, _ = evaluator(child_d)
    response['origem'] = 'd'
    response['is_st'] = is_dd_st
    data2.append(response)
    
    data.append([is_aa_st, is_bb_st, is_cc_st, is_dd_st, aa_cost, bb_cost, cc_cost, dd_cost])

500
Partition <{44, 478}> Partition <{44, 13}>
Partition <{474, 173}> Partition <{474, 79}>
Partition <{44, 13}> Partition <{44, 478}>
Partition <{16, 125}> Partition <{483, 196}>
Partition <{305, 78}> Partition <{315, 125}>
Partition <{451, 287}> Partition <{451, 351}>
Partition <{336, 302}> Partition <{336, 164}>
Partition <{293, 117}> Partition <{488, 432}>
Partition <{392, 426}> Partition <{104, 146}>
Partition <{41, 107}> Partition <{41, 391}>
Partition <{208, 191}> Partition <{351, 391}>
Partition <{427, 428}> Partition <{428, 125}>
Partition <{345, 357}> Partition <{16, 357}>
Partition <{380, 356}> Partition <{380, 238}>
Partition <{41, 469}> Partition <{340, 469}>
Partition <{208, 191}> Partition <{452, 31}>
Partition <{351, 391}> Partition <{41, 397}>
Partition <{112, 145}> Partition <{145, 397}>
Partition <{13, 478}> Partition <{44, 13}>
Partition <{360, 13}> Partition <{404, 214}>
Partition <{44, 13}> Partition <{13, 478}>
Partition <{392, 41}> Partition <{329, 493}>
Partiti

In [6]:
%%time

col = ['is_aa_st', 
    'is_bb_st', 
    'is_cc_st' ,
    'is_dd_st',
    'a_cost', 
    'b_cost', 
    'c_cost',
    'd_cost'
    ]

frame = pd.DataFrame(data, columns=col)

frame['less_equal_both_c'] = (frame['c_cost'] <= frame['a_cost']) & (frame['c_cost'] <= frame['b_cost'])
frame['less_equal_both_d'] = (frame['d_cost'] <= frame['a_cost']) & (frame['d_cost'] <= frame['b_cost'])

frame['less_both_c'] = (frame['c_cost'] < frame['a_cost']) & (frame['c_cost'] < frame['b_cost'])
frame['less_both_d'] = (frame['d_cost'] < frame['a_cost']) & (frame['d_cost'] < frame['b_cost'])

# frame['less_equal_a'] = (frame['c_cost'] <= frame['a_cost'])
# frame['less_equal_b'] = (frame['c_cost'] <= frame['b_cost'])

frame['diff_a_c'] = frame['a_cost'] - frame['c_cost']
frame['diff_b_c'] = frame['b_cost'] - frame['c_cost']

frame['diff_a_d'] = frame['a_cost'] - frame['d_cost']
frame['diff_b_d'] = frame['b_cost'] - frame['d_cost']

frame.head()

Wall time: 46 ms


,is_aa_st,is_bb_st,is_cc_st,is_dd_st,a_cost,b_cost,c_cost,d_cost,less_equal_both_c,less_equal_both_d,less_both_c,less_both_d,diff_a_c,diff_b_c,diff_a_d,diff_b_d
0,True,True,False,True,1953,1988,1916,1921,True,True,True,True,37,72,32,67
1,True,True,True,True,1875,1936,1864,1929,True,False,True,False,11,72,-54,7
2,True,True,True,True,1900,1953,1898,1940,True,False,True,False,2,55,-40,13
3,True,True,True,True,1919,1970,1910,1965,True,False,True,False,9,60,-46,5
4,True,True,False,True,1958,2039,1944,2016,True,False,True,False,14,95,-58,23


In [7]:

# all solutions were classified as steiner tree

# np.all(frame['is_aa_st'])
# np.all(frame['is_bb_st'])
# np.all(frame['is_cc_st'])

frame[['is_aa_st',	'is_bb_st',	'is_cc_st', 'is_dd_st']].describe()

,is_aa_st,is_bb_st,is_cc_st,is_dd_st
count,50,50,50,50
unique,1,1,2,2
top,True,True,True,True
freq,50,50,40,44


In [8]:
frame[['a_cost', 'b_cost','c_cost', 'd_cost']].describe()

,a_cost,b_cost,c_cost,d_cost
count,50.00000,50.000000,50.000000,50.000000
mean,1949.68000,1954.180000,1911.440000,1914.800000
std,41.17927,38.825455,68.364915,62.514978
min,1846.00000,1878.000000,1769.000000,1771.000000
25%,1924.75000,1930.000000,1870.250000,1878.250000
50%,1959.00000,1953.500000,1932.500000,1921.000000
75%,1978.75000,1985.750000,1960.000000,1957.750000
max,2034.00000,2039.000000,2013.000000,2016.000000


In [9]:
frame[['diff_a_c', 'diff_b_c']].describe()

,diff_a_c,diff_b_c
count,50.000000,50.000000
mean,38.240000,42.740000
std,44.814611,73.784521
min,0.000000,-85.000000
25%,8.250000,-9.250000
50%,18.000000,40.000000
75%,46.500000,72.000000
max,159.000000,220.000000


In [10]:
frame[['diff_a_d', 'diff_b_d']].describe()

,diff_a_d,diff_b_d
count,50.000000,50.000000
mean,34.880000,39.380000
std,61.619517,47.450894
min,-94.000000,0.000000
25%,-21.250000,8.250000
50%,39.500000,20.000000
75%,79.500000,60.750000
max,165.000000,168.000000


In [11]:
frame2 = pd.DataFrame(data2)

frame2.head()

,has_cycle,all_terminals_in,all_leaves_are_terminals,all_edges_are_reliable,graph_is_connected,origem,is_st
0,False,True,True,True,True,a,True
1,False,True,True,True,True,b,True
2,False,True,False,True,True,c,False
3,False,True,True,True,True,d,True
4,False,True,True,True,True,a,True


In [12]:
frame2[~frame2['is_st']]

,has_cycle,all_terminals_in,all_leaves_are_terminals,all_edges_are_reliable,graph_is_connected,origem,is_st
2,False,True,False,True,True,c,False
18,False,True,False,True,True,c,False
30,False,True,False,True,True,c,False
31,False,True,False,True,True,d,False
70,False,True,False,True,True,c,False
74,False,True,False,True,True,c,False
75,False,True,False,True,True,d,False
90,False,True,False,True,True,c,False
91,False,True,False,True,True,d,False
118,False,True,False,True,True,c,False
